In [2]:
from IPython.display import Image
import ee
import ee.mapclient
import datetime

ee.Initialize()
L1_AGBPSeasonals = ee.ImageCollection("projects/fao-wapor/L1_AGBP")
ETaCollection = ee.ImageCollection("projects/fao-wapor/L1_AET")
AET250 = ee.ImageCollection("users/lpeiserfao/AET250")

In [8]:
region = [[-25.0, -37.0], [60.0, -41.0], [58.0, 39.0], [-31.0, 38.0], [-25.0, -37.0]]

# Inizio calcolo WP dekad

### Above Ground Biomass Production with masked NoData (pixel < 0) and summed

In [9]:
L1_AGBPSeasonalMasked =L1_AGBPSeasonals.map(lambda lista: lista.updateMask(lista.gte(0)))

#the multiplier will need to be applied on net FRAME delivery, not on sample dataset
L1_AGBPSummedYearly = L1_AGBPSeasonalMasked.sum(); #.multiply(10); 

### Actual Evapotranspiration with valid ETa values (>0 and <254)

In [10]:
ETaColl1 = AET250.map(lambda immagine: immagine.updateMask(immagine.lt(254) and (immagine.gt(0)))) 

### add image property (days in dekad) as band

In [11]:
ETaColl2 = ETaColl1.map(lambda immagine: immagine.addBands(immagine.metadata('days_in_dk')));

### get ET value, divide by 10 (as per FRAME spec) to get daily value, and multiply by number of days in dekad summed annually

In [12]:
ETaColl3 = ETaColl2.map(lambda immagine: immagine.select('b1').divide(10).multiply(immagine.select('days_in_dk'))).sum()

### scale ETsum from mm/m² to m³/ha for WP calculation purposes

In [13]:
ETaTotm3 = ETaColl3.multiply(10)

### calculate biomass water productivity and add to map

In [23]:
WPbm = L1_AGBPSummedYearly.divide(ETaTotm3)
WPbm.getInfo()

{u'bands': [{u'crs': u'EPSG:4326',
   u'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   u'data_type': {u'precision': u'double', u'type': u'PixelType'},
   u'id': u'b1'}],
 u'type': u'Image'}

In [36]:
region = [[-25.0, -37.0], [60.0, -41.0], [58.0, 39.0], [-31.0, 38.0], [-25.0, -37.0]]
scala = WPbm.projection().nominalScale().getInfo()

visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region":region,
             "scale" : scala}

In [37]:
url_WPbm = WPbm.getThumbUrl(visparams)
Image(url=url_WPbm)

In [29]:
downConfig = {'scale': scala,
              'maxPixels' : 1500000000,
              'region': region};

fileName = 'WP_py'
export = ee.batch.Export.image(WPbm, fileName,downConfig);
#export.start();